In [4]:
import os
import glob
import numpy as np
#from ipynb.fs.full.corinvfuncs import *


In [6]:
#### USER INPUT ####

# Example 
poli='_VH'
paramsnpy = '/data/pmb229/isce/Somalia/T145/params.npy'
bbox = '8.65 10.85 46.6 49.28' # NEED TO CHANGE BBOX FOR EACH FRAME 
r = 4  # rlooks
a = 4  # alooks





# get params, decide ints
wd     = os.getcwd()
params = np.load(paramsnpy, allow_pickle=True).item()
locals().update(params)
%run decide_ints_stack.ipynb

geodir = workdir+'geo'+poli+'/'
resdir = workdir+'results_dates'+poli+'/'


/home/pmb229/jupyternotebook/corinvert_python


In [7]:
# define function 

def geocoder(file,pol,resdir,bbox,dem,geodir):
    print('geocoding '+file+' '+pol)
    tmpfile = resdir+'tmp'
    resfile = resdir+file
    os.chdir(resdir)
    
    command = 'imageMath.py -e \'a\' -o '+ tmpfile +' --a=\'' + resfile+';' + str(newnx)+';float;1;BSQ\''

    ! $command
    ! cp $resfile tmpfile
    
    command = 'geocodeIsce.py -f '+tmpfile+' -b \''+ str(bbox)+'\' -d '+dem+' -m '+workdir+'master -s '+workdir+'master -r '+str(rlooks)+' -a '+str(alooks)
    ! $command
    
    command = 'chmod 777 '+ tmpfile + '*'
    ! $command
    
    command = 'mv '+tmpfile +'.geo ' + resfile + '.geo'
    ! $command
    command = 'mv '+tmpfile +'.geo.xml ' + resfile + '.geo.xml'
    ! $command
    command = 'mv '+tmpfile +'.geo.vrt ' + resfile + '.geo.vrt'
    ! $command
    
    command = 'fixImageXml.py -i '+resfile+'.geo -b'
    ! $command
    
    os.chdir(workdir)
    

In [8]:
if not os.path.isdir(geodir):
    print('creating directory '+geodir)
    ! mkdir $geodir

creating directory /data/pmb229/isce/Somalia/T145/geo_VH/


In [11]:
filelist  = glob.glob(resdir + '*cor')
nfiles    = len(filelist)

for i in range(nfiles):
    file = os.path.basename(filelist[i])
    if os.path.isfile(resdir+file+'.geo') | os.path.isfile(geodir+file+'.geo'):
        print(file + ' already geocoded')
    else:
        geocoder(file,pol,resdir,bbox,dempath,geodir)
        command = 'mv '+filelist[i]+'.geo*'+ ' '+geodir
        ! $command
        


perm_20201118_20201130.cor already geocoded
perm_20201013_20201025.cor already geocoded
cmin.cor already geocoded
rel_20200826.cor already geocoded
rel_20201130.cor already geocoded
rel_20210210.cor already geocoded
perm_20200907_20200919.cor already geocoded
perm_20210129_20210210.cor already geocoded
perm_20200709_20200721.cor already geocoded
rel_20200907.cor already geocoded
rel_20201025.cor already geocoded
perm_20210210_20210222.cor already geocoded
perm_20201130_20201212.cor already geocoded
rel_20201212.cor already geocoded
perm_20200826_20200907.cor already geocoded
perm_20201106_20201118.cor already geocoded
rel_20201013.cor already geocoded
perm_20201025_20201106.cor already geocoded
perm_20210117_20210129.cor already geocoded
rel_20201106.cor already geocoded
perm_20201001_20201013.cor already geocoded
perm_20200919_20201001.cor already geocoded
rel_20201224.cor already geocoded
rel_20201001.cor already geocoded
rel_20210129.cor already geocoded
rel_20200709.cor already geo

In [12]:
if os.path.isfile(geodir+'rows.geo'):
    print('rows already geocoded')
else:
    rowfile = resdir+'rows'
    fid = open(rowfile, 'w')
    ! chmod 777 $rowfile
    for i in range(1,newny+1):
        rs  = np.ones((1,newnx)).flatten()*i
        rs  = rs.astype('float32')
        rs.tofile(fid)
    fid.close()
    geocoder('rows','',resdir,bbox,dempath,geodir)
    command = 'mv '+rowfile+'.geo* '+geodir
    ! $command
    
if os.path.isfile(geodir+'cols.geo'):
    print('cols already geocoded')
else:
    colfile = resdir+'cols'
    fid2 = open(colfile, 'w')
    ! chmod 777 $colfile
    for i in range(newny):
        rs2  = np.array(list(range(1,newnx+1))).astype('float32')
        rs2.tofile(fid2)
    fid2.close()
    geocoder('cols','',resdir,bbox,dempath,geodir)
    command = 'mv '+colfile+'.geo* '+geodir
    ! $command

if os.path.isfile(resdir+'tmp'):
    command ='rm '+resdir+'tmp*'
    ! $command

geocoding rows 
args:  Namespace(debug=False, dtype='float', equation='a', hh=None, noxml=False, out='/data/pmb229/isce/Somalia/T145/results_dates_VH/tmp', scheme='BSQ')
files:  ['--a=/data/pmb229/isce/Somalia/T145/results_dates_VH/rows;4541;float;1;BSQ']
GDAL open (R): /data/pmb229/isce/Somalia/T145/results_dates_VH/tmp.vrt
GDAL close: /data/pmb229/isce/Somalia/T145/results_dates_VH/tmp.vrt
2021-06-06 16:02:36,358 - isce.topsinsar.runGeocode - INFO - Geocoding Image
Number of products to geocode:  1
Output:  /data/pmb229/isce/Somalia/T145/results_dates_VH/tmp.geo
API open (WR): /data/pmb229/isce/Somalia/T145/dem/dem.crop
API open (WR): /data/pmb229/isce/Somalia/T145/results_dates_VH/tmp.geo
GDAL open (R): /data/pmb229/isce/Somalia/T145/results_dates_VH/tmp.vrt
GDAL open (R): /data/pmb229/isce/Somalia/T145/dem/demLat_N08_N11_Lon_E046_E050.dem.wgs84.vrt
ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing

In [13]:
# start code look_geo.m

# get nx_geo, ny_geo
f = geodir + 'c0.cor.geo.vrt'
v = open(f, 'r')
v = v.readlines()
l1 = v[0]
qs = [i for i in range(len(l1)) if l1.startswith('\"', i)]

nx_geo = int(l1[qs[0]+1:qs[1]])
ny_geo = int(l1[qs[2]+1:qs[3]])


In [14]:
files  = glob.glob(resdir + '*.cor')
os.chdir(geodir)
for j in files:
    file    = os.path.basename(j)
    newfile = file[0:-4]+'_'+str(r)+'r_'+str(a)+'a.cor.geo'
    oldfile = geodir+file+'.geo'
    if not os.path.isfile(geodir+newfile):
        fidi = open(oldfile,'r')
        fidr = open(geodir+'rows.geo', 'r')
        fido = open(geodir+'tmp', 'w')
        
        for k in range(ny_geo):
            tmp1 = np.fromfile(fidi, dtype=np.float32, count=nx_geo)
            tmp2 = np.fromfile(fidr, dtype=np.float32, count=nx_geo)
            tmp1[tmp2==0] = np.nan
            tmp1.tofile(fido)
        command = 'mv tmp '+oldfile
        fidi.close()
        fidr.close()
        fido.close()
        
        command = 'gdal_translate '+oldfile+'.vrt '+oldfile+'.tif'
        ! $command
        
        command = 'looks.py -i '+oldfile+' -o '+newfile+' -r '+str(r)+' -a '+str(a)
        ! $command
        command = 'chmod 777 *'
        ! $command
    else:
        print('already looked down '+newfile)
        
os.chdir(workdir)

ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.
Input file size is 3216, 2640
0...10...20...30...40...50...60...70...80...90...100 - done.
Output filename : perm_20201118_20201130_4r_4a.cor.geo
GDAL open (R): /data/pmb229/isce/Somalia/T145/geo_VH/perm_20201118_20201130.cor.geo.vrt
ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.
API open (WR): perm_20201118_20201130_4r_4a.cor.geo
GDAL close: /data/pmb229/isce/Somalia/T145/geo_VH/perm_20201118_20201130.cor.geo.vrt
API close:  perm_20201118_20201130_4r_4a.cor.geo
API open (R): perm_20201118_20201130_4r_4a.cor.geo
API close:  perm_20201118_20201130_4r_4a.cor.geo
Writing geotrans to VRT for perm_20201118_20201130_4r_4a.cor.geo
ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to 

ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.
Input file size is 3216, 2640
0...10...20...30...40...50...60...70...80...90...100 - done.
Output filename : rel_20201025_4r_4a.cor.geo
GDAL open (R): /data/pmb229/isce/Somalia/T145/geo_VH/rel_20201025.cor.geo.vrt
ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.
API open (WR): rel_20201025_4r_4a.cor.geo
GDAL close: /data/pmb229/isce/Somalia/T145/geo_VH/rel_20201025.cor.geo.vrt
API close:  rel_20201025_4r_4a.cor.geo
API open (R): rel_20201025_4r_4a.cor.geo
API close:  rel_20201025_4r_4a.cor.geo
Writing geotrans to VRT for rel_20201025_4r_4a.cor.geo
ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.
Input file size is 3216, 2640
0...10...

GDAL close: /data/pmb229/isce/Somalia/T145/geo_VH/rel_20201106.cor.geo.vrt
API close:  rel_20201106_4r_4a.cor.geo
API open (R): rel_20201106_4r_4a.cor.geo
API close:  rel_20201106_4r_4a.cor.geo
Writing geotrans to VRT for rel_20201106_4r_4a.cor.geo
ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.
Input file size is 3216, 2640
0...10...20...30...40...50...60...70...80...90...100 - done.
Output filename : perm_20201001_20201013_4r_4a.cor.geo
GDAL open (R): /data/pmb229/isce/Somalia/T145/geo_VH/perm_20201001_20201013.cor.geo.vrt
ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.
API open (WR): perm_20201001_20201013_4r_4a.cor.geo
GDAL close: /data/pmb229/isce/Somalia/T145/geo_VH/perm_20201001_20201013.cor.geo.vrt
API close:  perm_20201001_20201013_4r_4a.cor.geo
API open (R): perm_20201001

API open (WR): rel_20200814_4r_4a.cor.geo
GDAL close: /data/pmb229/isce/Somalia/T145/geo_VH/rel_20200814.cor.geo.vrt
API close:  rel_20200814_4r_4a.cor.geo
API open (R): rel_20200814_4r_4a.cor.geo
API close:  rel_20200814_4r_4a.cor.geo
Writing geotrans to VRT for rel_20200814_4r_4a.cor.geo
ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.
Input file size is 3216, 2640
0...10...20...30...40...50...60...70...80...90...100 - done.
Output filename : perm_20200814_20200826_4r_4a.cor.geo
GDAL open (R): /data/pmb229/isce/Somalia/T145/geo_VH/perm_20200814_20200826.cor.geo.vrt
ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.
API open (WR): perm_20200814_20200826_4r_4a.cor.geo
GDAL close: /data/pmb229/isce/Somalia/T145/geo_VH/perm_20200814_20200826.cor.geo.vrt
API close:  perm_20200814_20200826

GDAL close: /data/pmb229/isce/Somalia/T145/geo_VH/rel_20210117.cor.geo.vrt
API close:  rel_20210117_4r_4a.cor.geo
API open (R): rel_20210117_4r_4a.cor.geo
API close:  rel_20210117_4r_4a.cor.geo
Writing geotrans to VRT for rel_20210117_4r_4a.cor.geo
ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.
Input file size is 3216, 2640
0...10...20...30...40...50...60...70...80...90...100 - done.
Output filename : perm_20201224_20210117_4r_4a.cor.geo
GDAL open (R): /data/pmb229/isce/Somalia/T145/geo_VH/perm_20201224_20210117.cor.geo.vrt
ERROR 4: Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.
API open (WR): perm_20201224_20210117_4r_4a.cor.geo
GDAL close: /data/pmb229/isce/Somalia/T145/geo_VH/perm_20201224_20210117.cor.geo.vrt
API close:  perm_20201224_20210117_4r_4a.cor.geo
API open (R): perm_20201224

In [15]:
# geocode lon/lat radar files 

lonfileold=workdir+'merged/geom_master/lon.rdr.full';
latfileold=workdir+'merged/geom_master/lat.rdr.full';
lonfilenew=workdir+'merged/geom_master/lon.rdr.4alks_15rlks.full'
latfilenew=workdir+'merged/geom_master/lat.rdr.4alks_15rlks.full'

command = 'looks.py -i '+lonfileold+' -o '+lonfilenew+' -r '+str(rlks)+' -a '+str(alks)
! $command
command = 'looks.py -i '+latfileold+' -o '+latfilenew+' -r '+str(rlks)+' -a '+str(alks)
! $command

Output filename : /data/pmb229/isce/Somalia/T145/merged/geom_master/lon.rdr.4alks_15rlks.full
GDAL open (R): /data/pmb229/isce/Somalia/T145/merged/geom_master/lon.rdr.full.vrt
API open (WR): /data/pmb229/isce/Somalia/T145/merged/geom_master/lon.rdr.4alks_15rlks.full
GDAL close: /data/pmb229/isce/Somalia/T145/merged/geom_master/lon.rdr.full.vrt
API close:  /data/pmb229/isce/Somalia/T145/merged/geom_master/lon.rdr.4alks_15rlks.full
API open (R): /data/pmb229/isce/Somalia/T145/merged/geom_master/lon.rdr.4alks_15rlks.full
API close:  /data/pmb229/isce/Somalia/T145/merged/geom_master/lon.rdr.4alks_15rlks.full
Writing geotrans to VRT for /data/pmb229/isce/Somalia/T145/merged/geom_master/lon.rdr.4alks_15rlks.full
Output filename : /data/pmb229/isce/Somalia/T145/merged/geom_master/lat.rdr.4alks_15rlks.full
GDAL open (R): /data/pmb229/isce/Somalia/T145/merged/geom_master/lat.rdr.full.vrt
API open (WR): /data/pmb229/isce/Somalia/T145/merged/geom_master/lat.rdr.4alks_15rlks.full
GDAL close: /data

In [16]:
os.chdir(wd)